<a href="https://colab.research.google.com/github/ovapichi/biomecanica-datos/blob/main/Plataformas_WL_Armado_tabla.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Texto de título predeterminado
import pandas as pd
from google.colab import drive
#from fpdf import FPDF
from datetime import datetime
import numpy as np
import os
import matplotlib.pyplot as plt
from google.colab import files
drive.mount('/content/drive')

abril_2023='/content/drive/Shareddrives/Ova/Vertiz/Evaluacion abril 2023/'
septiembre_2023='/content/drive/Shareddrives/Ova/Vertiz/Saltos sep 2023/'

direccion=septiembre_2023

Mounted at /content/drive


In [ ]:
#Dirección de la carpeta de drive normaliza los nombres de los saltos por si estan en mayusculas, solo se hace una vez


direccionCarpeta=direccion + "Datos crudos"
for nombreArchivo in os.listdir(direccionCarpeta):
    if nombreArchivo[-7:]=="ABK.dat":
      nombreNuevo=nombreArchivo[:-7]+"abk.dat"
      os.rename(direccionCarpeta+"/"+nombreArchivo,direccionCarpeta+"/"+nombreNuevo)
    elif nombreArchivo[-8:]=="CMJa.dat":
      nombreNuevo=nombreArchivo[:-8]+"abk.dat"
      os.rename(direccionCarpeta+"/"+nombreArchivo,direccionCarpeta+"/"+nombreNuevo)
    elif nombreArchivo[-8:]=="Cabk.dat":
      nombreNuevo=nombreArchivo[:-8]+"abk.dat"
      os.rename(direccionCarpeta+"/"+nombreArchivo,direccionCarpeta+"/"+nombreNuevo)
    elif nombreArchivo[-7:]=="CMJ.dat":
      nombreNuevo=nombreArchivo[:-7]+"cmj.dat"
      os.rename(direccionCarpeta+"/"+nombreArchivo,direccionCarpeta+"/"+nombreNuevo)
    elif nombreArchivo[-7:]==" SJ.dat":
      nombreNuevo=nombreArchivo[:-7]+" sj.dat"
      os.rename(direccionCarpeta+"/"+nombreArchivo,direccionCarpeta+"/"+nombreNuevo)
    print(nombreArchivo)

In [ ]:
'''Limpiar cada uno de los archivos y unirlos por jugador en un archivo "saltosJugador.csv" con salto llamadas ['tiempo','cmjaDer','cmjaIzq','cmjDer','cmjIzq','sjDer','sjIzq'] y guardarlos un una nueva carpeta'''
#Generar los .csv desde los .dat solo se hace una vez
#def limpiar(archivo):
tiempo=[]
segundos=3
frecuencia=1000
datos=pd.DataFrame({'cmjaDer':[0],'cmjaIzq':[0],'cmjDer':[0],'cmjIzq':[0],'sjDer':[0],'sjIzq':[0]})
datos.drop([0], axis=0, inplace=True)

#arma el largo del df
for i in range(segundos*frecuencia):
  tiempo.append(-0.002+i/frecuencia)
datos=datos.append(tiempo)
datos=datos.fillna(0)

def grabarDrive(nombre,dflisto):
    #borra las 2 primeras lineas que WL pone al pp
    dflisto=dflisto.drop([0,1], axis=0).reset_index(drop=True)
    #elegir columnas que quiero sacar
    dfFinal=dflisto[['cmjaDer','cmjaIzq','cmjDer','cmjIzq','sjDer','sjIzq']]
    #dirección de carpeta que se va a grabar el .csv

    rutaDestino=direccion + 'Procesado'+nombre+".csv"
    with open(rutaDestino, 'w') as f:
      dfFinal.to_csv(f, decimal=",")
    return

def armarArchivo(archivos):
  for archivo in archivos:
    df=pd.read_csv(archivo,on_bad_lines='skip',sep = " ",header=None, encoding="utf-16")
    df=df[0].str.split(pat = ' ', expand = True)
    df2=df[0].str.split(pat = "\t", expand = True)
    df3=df2[[0,3]]
    punto=archivo.rfind(".")
    punto=punto-3
    barra=archivo.rfind("/")
    nombre=archivo[barra:punto]
    #print(nombre)
    #df3=df3.replace(".",",")

    #dfFinal=df3.drop([0,1,2], axis=0, inplace=True)
    salto=archivo[-7:-4]

    if salto=="abk":
      derecha=(df3[0]=="Der.")
      izquierda=(df3[0]=="Izq.")
      datos['cmjaDer']=df3[3][derecha].dropna().reset_index(drop=True)
      datos['cmjaIzq']=df3[3][izquierda].dropna().reset_index(drop=True)
    elif salto=="cmj":
      derecha=(df3[0]=="Der.")
      izquierda=(df3[0]=="Izq.")
      datos['cmjDer']=df3[3][derecha].dropna().reset_index(drop=True)
      datos['cmjIzq']=df3[3][izquierda].dropna().reset_index(drop=True)
    elif salto==" sj":
      derecha=(df3[0]=="Der.")
      izquierda=(df3[0]=="Izq.")
      datos['sjDer']=df3[3][derecha].dropna().reset_index(drop=True)
      datos['sjIzq']=df3[3][izquierda].dropna().reset_index(drop=True)
  grabarDrive(nombre,datos)
  return
#la dirección de la carpeta donde estan los .dat
direccionCarpeta= direccion + 'Datos crudos/' + "Juli Puz"
listaArchivo=[]
listaDeTres=[]
a=0
for nombreArchivo in os.listdir(direccionCarpeta):
  if nombreArchivo != '.ipynb_checkpoints':
    listaArchivo.append(direccionCarpeta+"/"+nombreArchivo)
  listaArchivo.sort()
for i in listaArchivo:
  a+=1
  listaDeTres.append(i)
  if a==3:
    armarArchivo(listaDeTres)
    #print(listaDeTres)
    listaDeTres=[]
    a=0


<ipython-input-16-702faad98bbf>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  datos=datos.append(tiempo)


In [ ]:
#Procesar los .csv para armar el PDF final
#1) Pico máximo de caída (<3.5 por lado, <7 total,Entrenar stiffness)
#2) Indice CMJa/CMJ (Entrenar coord neuromuscular MMII y MMSS) 10crit/30ok
#3) Indice CMJ/SJ (Entrenar Ciclo de estiramiento acortamiento, pliometría) 10crit/30ok
#4) Pico doble en la subida presalto (Coordinación MMII)
#5) Pico doble en la caída postsalto (Caídas + Drop Jump)
def graficar(df,salto):
  df.plot("Tiempo",ls="dashed",lw=1, marker="")
  plt.title(f"Fuerza en función del tiempo del salto {salto}")
  plt.xlabel("Tiempo [s]")
  plt.ylabel("Fuerza [N]")
  plt.xlim(0,3)
  plt.ylim(0,2000)
  plt.grid()
def tiempos(datos):
  tiempoCmja=(datos['cmjaDer']<30) & (datos['cmjaIzq']<30)
  tiempoCmj=(datos['cmjDer']<30) & (datos['cmjIzq']<30)
  tiempoSj=(datos['sjDer']<30) & (datos['sjIzq']<30)
  saltoCmja=(((tiempoCmja.sum()/1000)**2)*(9.81/8))
  saltoCmj=(((tiempoCmj.sum()/1000)**2)*(9.81/8))
  saltoSj=(((tiempoSj.sum()/1000)**2)*(9.81/8))
  return saltoCmja,saltoCmj,saltoSj

def indices(salto1,salto2):
  indice=(salto1-salto2)/salto2
  return indice

dfBosco=pd.DataFrame(columns=['nombres'])
direccionCsv=direccion + 'Procesado'
nombres=[]
masa=[]
saltoCmja=[]
saltoCmj=[]
saltoSj=[]
indiceBrazos=[]
indiceElasticidad=[]
riesgoCmjaDer=[]
riesgoCmjaIzq=[]
riesgoCmjDer=[]
riesgoCmjIzq=[]
riesgoSjDer=[]
riesgoSjIzq=[]

#Calcular con np.gradient las derivadas de cmja para ver si antes y despues del salto hay más de un 0 (puntos de inflexión)
#ver los index del salto 0.3 antes del salto y 0.3 post salto

doblePicoC=[] #trabajar fluidez en los saltos, saltos repetitivos
doblePicoE=[] #trabajar stiffness
for nombreArchivo in os.listdir(direccionCsv):
  if nombreArchivo != '.ipynb_checkpoints':
    punto=nombreArchivo.rfind('.')
    datos= pd.read_csv(direccionCsv+'/'+nombreArchivo,decimal=",").dropna()
    datos['cmjaDer']=datos['cmjaDer'].astype(float)
    datos['cmjaIzq']=datos['cmjaIzq'].astype(float)
    datos['cmjDer']=datos['cmjDer'].astype(float)
    datos['cmjIzq']=datos['cmjIzq'].astype(float)
    datos['sjDer']=datos['sjDer'].astype(float)
    datos['sjIzq']=datos['sjIzq'].astype(float)

    #datos=pd.read_csv('/content/drive/Shareddrives/Ova/Vertiz/Winlaborat/Ezequiel Ciamparella .csv',decimal=",").dropna()
  #pasar estas 2 lineas a antes de grabarlos
    datos=datos.rename(columns={'Unnamed: 0':'Tiempo'})
    datos['Tiempo']=datos['Tiempo']/1000

    salto1,salto2,salto3=tiempos(datos)


    #(fza max caída * 0.07)/peso
    peso=(datos['cmjaDer'][0]+datos['cmjaIzq'][0]+datos['cmjaDer'][1]+datos['cmjaIzq'][1])/2
    riesgo1Der=100*(datos['cmjaDer'].max()*0.07)/peso
    riesgo1Izq=100*(datos['cmjaIzq'].max()*0.07)/peso
    riesgo2Der=100*(datos['cmjDer'].max()*0.07)/peso
    riesgo2Izq=100*(datos['cmjIzq'].max()*0.07)/peso
    riesgo3Der=100*(datos['sjDer'].max()*0.07)/peso
    riesgo3Izq=100*(datos['sjIzq'].max()*0.07)/peso
    nombres.append(nombreArchivo[0:punto])
    saltoCmja.append(salto1*100)
    saltoCmj.append(salto2*100)
    saltoSj.append(salto3*100)
    indiceB=indices(salto1,salto2)
    indiceBrazos.append(indiceB*100)
    indiceE=indices(salto2,salto3)
    indiceElasticidad.append(indiceE*100)
    riesgoCmjaDer.append(riesgo1Der)
    riesgoCmjaIzq.append(riesgo1Izq)
    riesgoCmjDer.append(riesgo2Der)
    riesgoCmjIzq.append(riesgo2Izq)
    riesgoSjDer.append(riesgo3Der)
    riesgoSjIzq.append(riesgo3Izq)
    masa.append(peso)


dfBosco['nombres']=nombres
dfBosco['peso']=masa
dfBosco['riesgoCmjaDer']=riesgoCmjaDer
dfBosco['riesgoCmjaIzq']=riesgoCmjaIzq
dfBosco['riesgoCmjDer']=riesgoCmjDer
dfBosco['riesgoCmjIzq']=riesgoCmjIzq
dfBosco['riesgoSjDer']=riesgoSjDer
dfBosco['riesgoSjIzq']=riesgoSjIzq
dfBosco['saltoCmja']=saltoCmja
dfBosco['saltoCmj']=saltoCmj
dfBosco['saltoSj']=saltoSj
dfBosco['indiceBrazos']=indiceBrazos
dfBosco['indiceElasticidad']=indiceElasticidad
#dfBosco=dfBosco.astype(int)
#print(f"{masa,riesgoCmjaDer,riesgoCmjaIzq,riesgoCmjDer,riesgoCmjIzq,riesgoSjDer,riesgoSjIzq,indiceBrazos,indiceElasticidad,saltoCmja,saltoCmj,saltoSj}")
#print(datos.max())
#graficar(datos,"Cmj")



dfBosco[['peso','riesgoCmjaDer','riesgoCmjaIzq','riesgoCmjDer','riesgoCmjIzq','riesgoSjDer','riesgoSjIzq','saltoCmja','saltoCmj','saltoSj','indiceBrazos','indiceElasticidad']]=dfBosco[['peso','riesgoCmjaDer','riesgoCmjaIzq','riesgoCmjDer','riesgoCmjIzq','riesgoSjDer','riesgoSjIzq','saltoCmja','saltoCmj','saltoSj','indiceBrazos','indiceElasticidad']].astype(int)
#agregar manualmente un jugador manualmente
dfBosco=dfBosco.append(pd.Series(['Bianca Romani',620,8,10,10,13,8,11,31,29,24,8,21],index=dfBosco.columns),ignore_index=True)
dfBosco

<ipython-input-17-62e759e3770c>:115: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfBosco=dfBosco.append(pd.Series(['Bianca Romani',620,8,10,10,13,8,11,31,29,24,8,21],index=dfBosco.columns),ignore_index=True)


,nombres,peso,riesgoCmjaDer,riesgoCmjaIzq,riesgoCmjDer,riesgoCmjIzq,riesgoSjDer,riesgoSjIzq,saltoCmja,saltoCmj,saltoSj,indiceBrazos,indiceElasticidad
0,Alejo Fuentes,750,10,8,9,9,7,12,35,33,27,7,21
1,Franco Filippo,948,9,9,8,9,8,8,21,19,17,9,12
2,Guadalupe Garavaglia,552,7,10,9,8,9,9,21,18,13,19,36
3,Joaquin Ulicki,691,12,16,12,12,10,12,43,35,29,24,19
4,Gino Khoury,922,9,10,9,8,8,8,24,24,22,0,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,zara gonzalez,632,8,9,11,7,9,6,16,16,10,-1,55
76,agustin massucco,868,9,9,20,13,16,15,38,32,14,18,128
77,ignacio sokei,609,10,12,13,12,12,10,52,14,38,276,-63
78,julieta pus,731,12,12,12,12,11,12,26,26,19,0,33


# Unir datasets


In [ ]:
df=dfBosco
#df2=pd.read_csv(direccion+'datos mcgill abril.csv')
'''Solo SEP 2023'''
df2=pd.read_excel(direccion +'Evaluacion vertiz.xlsx')

df['nombres']=df['nombres'].str.lower()

df2['nombre']=df2['nombre'].str.lower()

# SOLAMENTE PARA SEPTIEMBRE 2023
df.nombres=df.nombres.replace({'santino t':'santino troglio','sofia urbant':'sofia urbandt','fiama rojas':'fiamma rojas','federico testei':'federico testai','alejandro betancurt':'alejandro betancourt',
                               'ema de giorgi':'emma de giorgi','ezequien cufone':'ezequiel cufone','ova piccinini':'osvaldo garcia piccinini',
                               'thiago covalan':'thiago corvalan','santino cataneo':'santino cattaneo','pollo':'nicolas argenti','arashiro':'matias arashiro',
                               'federico li':'federico lee','maria pia orostizu':'pia gorostizu'})
df2.nombre=df2.nombre.replace({'santino troglio ':'santino troglio','julieta puz':'julieta pus','sofia urbant':'sofia urbandt','fiama rojas':'fiamma rojas','federico testei':'federico testai','alejandro betancurt':'alejandro betancourt',
                               'ema de giorgi':'emma de giorgi','ezequien cufone':'ezequiel cufone','ova piccinini':'osvaldo garcia piccinini',
                               'thiago covalan':'thiago corvalan','santino cataneo':'santino cattaneo','pollo':'nicolas argenti','arashiro':'matias arashiro',
                               'federico li':'federico lee','pia orostizu':'pia orostizu'})

'''df.nombres=df.nombres.replace({'alejando betancuort':'alejandro betancourt','emma di giorgi':'emma de giorgi','matian mendoza':'matias mendoza',
                               'juan cruz':'juan cruz trejo','maria pia':'maria pia gorostizu','nico pollo':'nicolas argenti',
                               'maximiliano harley':'maximiliano herley','fede macias':'federico macias','issac morales':'isaac morales'})
#df2.nombre=df2.nombre.replace({'agustina belén torres costa':'agustina torres','alanís marin':'alanis marin',
                               'carlos leonardo fernández':'carlos leonardo fernandez','claudia alejandra romano':'claudia romano',
                               'brandon david mamani sarzuri':'david mamani','francisco gonzalez laneve':'francisco gonzalez',
                               'elias joel alderete':'elias alderete','ignacio hiroki sokei':'ignacio sokei',
                               'matias ignacio mendoza':'matias mendoza','ema hernandez':'emma hernandez','ema muñoz':'emma munoz',
                               'federico benjamin berdasco castillo':'fede berdasco','elisa giordano micheletto':'elisa giordano',
                               'nicolás shaw':'nicolas shaw','nicole ayelen herley moreno':'nicole moreno','joaquin matias ulicki maceira':'joaquin ulicki',
                               'mia müller':'mia muller','julieta belen fiumara':'julieta fiumara','julieta puz':'julieta pus','mica caputo':'micaela caputo'})
'''
df3=df.merge(df2, left_on='nombres', right_on='nombre', how="outer")

df4=df3.drop(columns=['nombres'])
df5=df4.sort_values(by='nombre')

df6=df5[['categoria', 'rama','nombre','peso', 'anterior', 'posterior', 'izquierda', 'derecha', 'saltoCmja', 'saltoCmj','saltoSj','riesgoCmjaDer', 'riesgoCmjaIzq', 'riesgoCmjDer','riesgoCmjIzq', 'riesgoSjDer', 'riesgoSjIzq', 'indiceBrazos', 'indiceElasticidad']]
df6

,categoria,rama,nombre,peso,anterior,posterior,izquierda,derecha,saltoCmja,saltoCmj,saltoSj,riesgoCmjaDer,riesgoCmjaIzq,riesgoCmjDer,riesgoCmjIzq,riesgoSjDer,riesgoSjIzq,indiceBrazos,indiceElasticidad
77,mayores,M,agustin massucco,868,25,23,5,15,38,32,14,9,9,20,13,16,15,18,128
15,sub 21,F,agustina torres,783,71,76,34,38,13,12,9,9,6,8,7,8,7,4,42
16,sub 18,M,alan servian,828,32,39,13,16,25,22,15,15,11,15,11,15,14,14,40
17,sub 21,F,alanis marin,731,35,16,28,22,19,18,15,7,14,11,8,10,7,3,23
18,sub 18,M,alejandro betancourt,794,32,47,28,28,45,36,34,20,16,13,14,17,14,23,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14,Sub 18,M,tobias zabala,683,25,6,5,8,29,24,21,10,8,10,7,11,7,19,11
73,sub 16,M,tomas campos,589,45,38,16,7,25,21,20,12,15,16,10,16,10,16,6
74,sub 18,M,ulises oviedo,675,48,33,16,28,42,39,31,11,8,11,7,8,10,9,23
75,sub 18,F,valentina flores,616,43,78,32,36,24,20,18,9,7,8,8,7,11,18,11


# Exportar


In [ ]:
rutaFinal=direccion +"datosBosco"+".csv"
with open(rutaFinal, 'w') as f:
  df6.to_csv(f, decimal=",")

In [ ]:
  '''funciones para matplotlib'''
  #señalar puntos del gráfico "texto" "xy=(ubicación x, ubicación y)", xytext=(ubicación x y de lo que dice la anotación), shrink(distancia de la flecha del punto)
  #ax.annotate('máximo local', xy=(2, 1), xytext=(3, 1.5),
            #arrowprops=dict(facecolor='red', shrink=0.05))

  #trazar barras verticales
  #(ubicacion x, color, altrura de la linea inicio y fin)
  #ax.vlines(1, color="red", ymin=-1.5, ymax=1.2)
  #ax.vlines(3, color="pink", ymin=-0.5, ymax=0.5)

  # # agregar rectangulos de colores VERTICALES
  #axvspan = un span vertical en el eje X (an X vertical span)
  # # por defecto toma todo el eje Y
  #(inicio x, fin x, alpha=transparencia)
  #ax.axvspan(0.5, 1.5, color="green", alpha=0.1)

  # # axhspan = un span horizontal en el eje X (an X horizontal span)
  #RECTANGULOS HORIZONTALES
  # # por defecto toma todo el eje X
  #ax.axhspan(-1,1, color="brown", alpha=0.1)